# 0 摘要

本文提出FM模型（因子分解机），它是一种保留了SVM优点的因子分解机模型。相比于SVM，FM可以对所有的交叉特征使用因数分解参数，因此对于高维稀疏特征更优优势。在时间复杂度上，FM可以在线性时间复杂度完成训练和预测。同时，FM是一种通用模型，更易使用。

# 1 简介

SVM是一种常用的机器学习算法，但其面对高维稀疏特征空间中复杂kernel场景时无法学习到可靠的超平面。另一方面，因数分解类模型无法应用到标准的n维向量输入中，且对于特定模型需要特征的设计。

因此本文提出Factorization Machine模型，其具有以下优点：
- FM可以适用高维稀疏输入数据场景下的参数估计
- FM具有线性时间复杂度的训练和预测过程
- FM对于任何实向量都可以work，是一个通用算法模型

# 2 高维稀疏特征

在实际工业界，推荐搜索广告领域的相关数据往往是高维稀疏的，例如广告类别、素材、广告位、商品类别等，在经过One Hot Encoding会变的高维且稀疏。

我们定义$m(x)$为输入向量$x$中的非零元素个数，那么对于由多个输入向量构成的数据集$\mathcal{D}$，令$\bar{m}_D$代表数据集的平均非零元素个数，当$\bar{m}_D << n$时，我们称为高维稀疏数据。

# 3 Factorization Machine

FM实际上是在线性模型基础上引入了二阶交叉特征，是的模型能够学到高阶交叉信息。

## 3.1 FM模型

(1) 二阶FM（2-way FM）的公式：

$$\hat{y}(x)=w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n \langle v_i, v_j \rangle x_i x_j$$

其中$\langle v_i, v_j \rangle = \sum_{f=1}^k v_{i,f}\cdot v_{j, f}$

对于矩阵$V \in \mathbb{R}^{n\times k}$。矩阵每一行代表对应特征的一个$k$维隐向量

(2) Expressiveness

我们知道，对于任意正定矩阵$W$，当$k$最够大的情况下，存在一个矩阵$V$，满足$W = V\cdot V^T$，即一个$n \times n$的矩阵可以近似分解为两个$n \times k$的小矩阵。在实际中由于数据有限，$k$往往选择比较小。

因此限制$k$的话，从而限制了FM的表达，给模型带来了更好的泛化，并因此改善了稀疏性下的特征交叉参数矩阵的学习。

(3) Parameter Estimation Under Sparsity

在稀疏场景下，对于交叉特征的参数训练往往数据不足，而FM通过**break the independence of the interaction parameters**使得其可以很好地在稀疏场景下估计交叉特征参数。这是因为FM将交叉特征的参数估计相关联，对于一个交叉特征的参数估计，其可以帮助估计其他相关的交叉特征。

例如，假设我们有交叉特征gender=male & movie_genre=war，我们需要估计这个交叉特征前的参数$w_{male\ and\ war}$，FM通过将$w_{male\ and\ war}$分解为$\langle v_{male}, v_{war}\rangle$的方式进行估计，那么对于每次更新male或者war的隐向量$v$时，都会影响其他与male交叉的特征参数估计，以及和war交叉的特征参数估计。

另外，FM这种矩阵分解的方式，使得泛化性能更好，对于train data中没出现过的交叉项，FM仍然可以通过向量内积给出较好的预估。

(4) Computation

FM在线性模型的基础上增加了二阶交乘项特征，令隐向量维度为$k$，特征数为$n$，则FM时间复杂度为$O(kn^2)$，但实际上通过化简可以使得时间复杂度降到$O(kn)$

![](pictures/fm-equation.png)

> FM的化简思路实际为$ab=\frac{1}{2}(a+b)^2-\frac{1}{2}(a^2+b^2)$

另外，FM在稀疏场景下的时间复杂度为$O(k \bar{m}_D)$，例如FM的特例（只有user和item信息）MF模型中$\bar{m}_D=2$

## 3.2 FM as Predictors

FM可以用在不同的预测任务上：
- Regression
- Binary classification
- Ranking

> 在实际使用中，往往需要在目标函数加入$L2$正则化方式防止过拟合

## 3.3 Learning FM

FM中各个参数的梯度计算：

![](pictures/fm-gradients.png)

> FM的梯度计算时间复杂度为$O(1)$，可以看出FM本身的梯度更新也可以在线性时间完成

## 3.4 d-way FM

除了2-way FM外，还有融合了高阶交叉特征组合的FM，其也可以通过数学化简到线性的计算时间复杂度。

## 3.5 总结

FM通过使用矩阵分解（向量内积）的方式来估计任意两个交叉特征的参数，具有以下优点：
- 适用于高维稀疏场景的估计，并且可以对训练集中未出现的交叉信息也可以给出相对合理的参数估计
- FM可以在线性时间复杂度完成估计和学习，使得SGD优化可行